In [ ]:
import requests
import pandas as pd
import json
from io import StringIO
import time

def fetch_oecd_data(metric):
    url = f"https://sdmx.oecd.org/public/rest/data/{metric}/all?startPeriod=2000&endPeriod=2025&dimensionAtObservation=AllDimensions&format=csvfilewithlabels"
    response = requests.get(url)
    print(response.status_code)
    # digesting the csv text into a pandas df
    df = pd.read_csv(StringIO(response.text)) #stringio makes it "appear" like a csv to pandas
    #df = df[['TIME_PERIOD', 'Reference area', 'REF_AREA', 'Measure','OBS_VALUE']] # selecting columns to keep
    return df

TradeUnion_code = 'OECD.ELS.SAE,DSD_TUD_CBC@DF_TUD,'
CorpTax_code = 'OECD.CTP.TPS,DSD_TAX_CIT@DF_CIT,1.0/.A..ST..S13+S1311+S13M..'
GovSpend_code = 'OECD.SDD.NAD,DSD_NASEC10@DF_TABLE11,/A..S13...OTE.....V..'

df = fetch_oecd_data(GovSpend_code)
print(df.shape)

In [70]:
def split_gov_expenditure(df, sectors):
    df_dict = {}
    for sector in sectors:
        df_dict[sector] = df.loc[df['Expenditure'] == sector][['Reference area', 'REF_AREA', 'TIME_PERIOD', 'OBS_VALUE', 'UNIT_MULT', 'CURRENCY']]
    return df_dict

govspend_df = split_gov_expenditure(df, ['Education', 'Health', 'Housing', 'Community development'])
govspend_df

{'Education':       Reference area REF_AREA  TIME_PERIOD  OBS_VALUE  UNIT_MULT CURRENCY
 616        Australia      AUS         2016   100934.0          6      AUD
 617        Australia      AUS         2015    97076.0          6      AUD
 618        Australia      AUS         2014    94175.0          6      AUD
 619        Australia      AUS         2013    92229.0          6      AUD
 620        Australia      AUS         2012    90839.0          6      AUD
 ...              ...      ...          ...        ...        ...      ...
 62135         Norway      NOR         2021   208296.0          6      NOK
 62136         Norway      NOR         2022   221661.0          6      NOK
 62137         Norway      NOR         2023   235035.0          6      NOK
 62268         Norway      NOR         2000    78493.0          6      NOK
 62269         Norway      NOR         2001    85387.0          6      NOK
 
 [865 rows x 6 columns],
 'Health':       Reference area REF_AREA  TIME_PERIOD  OBS_V

In [71]:
def create_exchrate_dict(countries):
    dct = {}
    for country in countries:
        url = f'https://api.worldbank.org/v2/country/{country}/indicator/PA.NUS.FCRF?format=json&per_page=300&fl=country'
        response = requests.get(url).json()
        dct[country] = {}
        if len(response) > 1 and isinstance(response[1], list):
            for row in response[1]:
                dct[country][row['date']] = row['value']
        time.sleep(1)
    return dct
exchrate_dict = create_exchrate_dict(list(set(govspend_df['Education']['REF_AREA'])))

In [72]:
for area in govspend_df.keys():
    converted_values = []
    for index, row in govspend_df[area].iterrows():
        exchrate = exchrate_dict[row['REF_AREA']].get(str(row['TIME_PERIOD']))
        if exchrate != None:
            converted_values.append(row['OBS_VALUE']/exchrate)
        else:
            converted_values.append(None)
    govspend_df[area]['OBS_USD'] = converted_values

In [73]:
govspend_df

{'Education':       Reference area REF_AREA  TIME_PERIOD  OBS_VALUE  UNIT_MULT CURRENCY  \
 616        Australia      AUS         2016   100934.0          6      AUD   
 617        Australia      AUS         2015    97076.0          6      AUD   
 618        Australia      AUS         2014    94175.0          6      AUD   
 619        Australia      AUS         2013    92229.0          6      AUD   
 620        Australia      AUS         2012    90839.0          6      AUD   
 ...              ...      ...          ...        ...        ...      ...   
 62135         Norway      NOR         2021   208296.0          6      NOK   
 62136         Norway      NOR         2022   221661.0          6      NOK   
 62137         Norway      NOR         2023   235035.0          6      NOK   
 62268         Norway      NOR         2000    78493.0          6      NOK   
 62269         Norway      NOR         2001    85387.0          6      NOK   
 
             OBS_USD  
 616    75031.929343  
 61

In [74]:
def create_gdp_dict(countries):
    dct = {}
    for country in countries:
        url = f'https://api.worldbank.org/v2/country/{country}/indicator/NY.GDP.MKTP.CD?format=json&per_page=300&fl=country'
        response = requests.get(url).json()
        dct[country] = {}
        if len(response) > 1 and isinstance(response[1], list):
            for row in response[1]:
                dct[country][row['date']] = row['value']
        time.sleep(1)
    return dct
gdp_dict = create_gdp_dict(list(set(govspend_df['Education']['REF_AREA'])))

In [78]:
gdp_dict

{'GRC': {'2024': None,
  '2023': 243498333237.802,
  '2022': 218880566444.349,
  '2021': 218303801895.326,
  '2020': 191362985554.861,
  '2019': 207305649887.019,
  '2018': 213298873494.394,
  '2017': 200381103984.298,
  '2016': 193097239005.87,
  '2015': 194567373678.099,
  '2014': 233911581521.062,
  '2013': 236556279640.705,
  '2012': 238841140018.233,
  '2011': 283228079776.041,
  '2010': 296417644404.034,
  '2009': 326829054686.306,
  '2008': 351121399546.485,
  '2007': 314226996944.108,
  '2006': 269073415333.676,
  '2005': 242315668619.405,
  '2004': 234979615898.225,
  '2003': 196930509813.384,
  '2002': 150253800085.865,
  '2001': 132050474720.03,
  '2000': 125760166224.805,
  '1999': 137131371955.307,
  '1998': 139612812175.718,
  '1997': 138766067640.085,
  '1996': 142502984144.96,
  '1995': 134974613913.811,
  '1994': 114980063202.247,
  '1993': 107295704518.43,
  '1992': 114608178405.434,
  '1991': 103680863712.844,
  '1990': 96529587274.2906,
  '1989': 78067933277.3815,
 

In [108]:
for area in govspend_df.keys():
    converted_values = []
    for index, row in govspend_df[area].iterrows():
        ref_area = row['REF_AREA']
        time_period = row['TIME_PERIOD']
        gdp = gdp_dict[ref_area].get(str(time_period))
        if gdp != None:
            expenditure_share = row['OBS_USD']*1000000/gdp
            converted_values.append(expenditure_share)
        else:
            converted_values.append(None)
    govspend_df[area]['OBS_SHARE_GDP'] = converted_values
govspend_df

{'Education':       Reference area REF_AREA  TIME_PERIOD  OBS_VALUE  UNIT_MULT CURRENCY  \
 616        Australia      AUS         2016   100934.0          6      AUD   
 617        Australia      AUS         2015    97076.0          6      AUD   
 618        Australia      AUS         2014    94175.0          6      AUD   
 619        Australia      AUS         2013    92229.0          6      AUD   
 620        Australia      AUS         2012    90839.0          6      AUD   
 ...              ...      ...          ...        ...        ...      ...   
 62135         Norway      NOR         2021   208296.0          6      NOK   
 62136         Norway      NOR         2022   221661.0          6      NOK   
 62137         Norway      NOR         2023   235035.0          6      NOK   
 62268         Norway      NOR         2000    78493.0          6      NOK   
 62269         Norway      NOR         2001    85387.0          6      NOK   
 
             OBS_USD  OBS_SHARE_GDP  
 616    750

In [109]:
df_health = govspend_df['Health']
df_health.loc[(df_health['REF_AREA'] == 'NOR')&(df_health['TIME_PERIOD'] == 2001)]

,Reference area,REF_AREA,TIME_PERIOD,OBS_VALUE,UNIT_MULT,CURRENCY,OBS_USD,OBS_SHARE_GDP
61543,Norway,NOR,2001,111758.0,6,NOK,12429.081227,0.071333


In [110]:
12429 / (gdp_dict['NOR']['2001']/1000000)

0.0713329090679308